In [ ]:
!pip install transformers torch ipython

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from IPython.display import display, HTML

In [3]:
repo = "nqzfaizal77ai/mt-230m-pretrained-en-1bc-exp"

In [2]:
repo = "Qwen/Qwen3-0.6B"

In [3]:
precision = 'full' # @param ["half", "full"]
if precision == "half":
    selected_precision = torch.float16
elif precision == "full":
    selected_precision = torch.float32

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
model = AutoModelForCausalLM.from_pretrained(repo, torch_dtype=selected_precision)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(repo)

In [6]:
# @title Hyperparameter Inference
max_length = 100  # @param {type:"integer"} (maximum length of generated text)
min_length = 10   # @param {type:"integer"} (minimum length should be less than max_length)
top_k = 50        # @param {type:"integer"} (typical range 0-100)
top_p = 0.92      # @param {type:"number"} (should be <= 1.0, typically 0.7-0.95)
temperature = 0.7  # @param {type:"number"} (typically 0.1-1.0, higher = more random)
repetition_penalty = 1.2  # @param {type:"number"} (typically 1.0-1.5)
do_sample = True   # @param {type:"boolean"}
use_cache = True   # @param {type:"boolean"}

In [7]:
prompt = "The person who likes to incite others has a weakness"

Stochastic Decoding

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt").to(device)
output = model.generate(
                        inputs["input_ids"],
                        max_length=max_length,
                        min_length=min_length,
                        # num_return_sequences=1,
                        top_k=top_k,
                        top_p=top_p,
                        temperature=temperature,
                        repetition_penalty=repetition_penalty,
                        # num_beams=5,
                        # length_penalty=0.8,
                        # penalty_alpha=0.5,
                        do_sample=True,
                        use_cache=True
                        )
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

Greedy Decoding

In [8]:
# prompt: make greedy decoding code
inputs = tokenizer(prompt, return_tensors="pt").to(device)
# Greedy Decoding
greedy_output = model.generate(
    inputs["input_ids"],
    max_length=max_length,
    min_length=min_length,
    do_sample=False  # Set do_sample to False for greedy decoding
)
decoded_output = tokenizer.decode(greedy_output[0], skip_special_tokens=True)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
from IPython.display import HTML, display

# Replace newlines with <br>
formatted_output = decoded_output.replace("\n", "<br>")

# HTML and CSS for displaying the output in a square box
html_code = f"""
<div style='border: 2px solid black; padding: 10px; width: 500px; height: 300px; overflow: auto;'>
    <p style='font-family: monospace;'>{formatted_output}</p>
</div>
"""

# Display the styled output
display(HTML(html_code))